In [1]:
import xgboost as xgb
from xgboost import XGBRegressor
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error,r2_score, mean_absolute_error
from sklearn.model_selection import cross_val_score,train_test_split
from math import sqrt
import matplotlib.dates as mdates
from sklearn.preprocessing import StandardScaler,OrdinalEncoder
from sklearn.model_selection import GridSearchCV, KFold
import os

In [11]:
# 设置全局控制的变量
freq = 'D'
region = 'NA'
model_sty = 'xgb'

if freq == 'H':
    data_num = 17520
    list_col = ['NO2+h','NO2-h','NO2-2h','NO2-3h']
    col_name = ['NO2+h_rfr','NO2+h_xgb','NO2+h_resnet']
    col_name  = list_col[0] + '_' + model_sty
    file_name = 'pol_mete_' + region + '_' + freq + '.csv'
    file_name_pre = 'pol_mete_' + region + '_' + freq + '_predict.csv'
if freq == '3H':
    data_num = 5840
    list_col = ['NO2+3h','NO2-3h','NO2-6h','NO2-9h']
    col_name  = list_col[0] + '_' + model_sty
    file_name = 'pol_mete_' + region + '_' + freq + '.csv'
    file_name_pre = 'pol_mete_' + region + '_' + freq + '_predict.csv'
if freq == 'D':
    data_num = 730
    list_col = ['NO2+d','NO2-d','NO2-2d','NO2-3d']
    col_name  = list_col[0] + '_' + model_sty
    file_name = 'pol_mete_' + region + '_' + freq + '.csv'
    file_name_pre = 'pol_mete_' + region + '_' + freq + '_predict.csv'

file_path = 'data/'
file_path_model = ''

## Ten time-steps

### Load data

In [5]:
# load data with 10 time-steps
data = np.load(file_path + 'numpy/NA_d_array_14.npz',allow_pickle=True, mmap_mode='r')
xtrain,ytrain,xtest,ytest,ytrain_,ytest_ = data['a'][:,-10:,:],data['b'],data['c'][:,-10:,:],data['d'],data['e'],data['f']

In [ ]:
xtrain.shape, xtrain.dtype

In [7]:
# flatten
xtrain, xtest = xtrain.reshape(-1, 360), xtest.reshape(-1, 360)

### Cross validation

In [13]:
# 函数：XGBoost模型 define XGBoost model
def model_xgbr_cv(xtrain, ytrain, xtest, ytest,num_round,verbosity,eta,gamma,colsample_bynode
               ,colsample_bytree,min_chlid_weight,alpha,lambdas,max_depth=10):

    # 建模：XGBRegressor
    xgbr = XGBRegressor(verbosity=verbosity,learning_rate=eta,gamma=gamma,colsample_bynode=colsample_bynode
                        ,colsample_bytree=colsample_bytree, min_chlid_weight=min_chlid_weight,alpha=alpha
                        ,lambdas=lambdas, n_estimators=num_round,max_depth=max_depth
                        , tree_method='gpu_hist', gpu_id=0, single_precision_histogram=True
                        # , subsample=0.1, sampling_method="gradient_based"
                        #,n_jobs=-1
                       ).fit(xtrain, ytrain)

    feature_importances = xgbr.feature_importances_
    #feature_importances = pd.DataFrame([*zip(xtrain.columns,feature_importances)])
    #feature_importances = feature_importances.sort_values(by=1,axis=0,ascending=False)
    #print(feature_importances)
    
    # 训练集预测结果和评价指标
    predict = xgbr.predict(xtrain)
    RMSE = sqrt(mean_squared_error(ytrain,predict))
    MAE = mean_absolute_error(ytrain, predict)
    R2 = r2_score(ytrain,predict)
    # 测试集预测结果和评价指标
    ypredict = xgbr.predict(xtest)
    val_RMSE = sqrt(mean_squared_error(ytest,ypredict))
    val_MAE = mean_absolute_error(ytest, ypredict)
    val_R2 = r2_score(ytest,ypredict)
    print('模型在训练集表现','RMSE:',RMSE,'R2:',R2)
    print('模型在测试集表现','val_RMSE:',val_RMSE,'val_R2:',val_R2)
    return RMSE,MAE,R2,val_RMSE,val_MAE,val_R2,xgbr,feature_importances

In [15]:
# cross validation
Training = False
kfold = KFold(n_splits=5, shuffle=False)
results_cv = []
preds_list = []
for i, j in enumerate(kfold.split(xtrain, ytrain)):
    idx_train, idx_test = j[0], j[1]
    save_model_path = os.path.join(file_path_model, 'model', model_sty)
    if not os.path.exists(save_model_path):
        print("creating path: " + save_model_path)
        os.mkdir(save_model_path)

    if Training:
        result_ = model_xgbr_cv(xtrain=xtrain[idx_train], ytrain=ytrain[idx_train], xtest=xtrain[idx_test], ytest=ytrain[idx_test], verbosity=0, eta=0.19
                            ,num_round=295, gamma=0.25, colsample_bynode=1, colsample_bytree=1, min_chlid_weight=0.5, alpha=1.9, lambdas=1.0)
        # save model
        xgbr = result_[6]
        joblib.dump(xgbr, save_model_path + '/model_xgboost_cv_' + str(i) +'.dat')
        results_cv.append([result_[3], result_[4], result_[5]])
    else:
        xgbr = joblib.load(save_model_path + '/model_xgboost_cv_' + str(i) +'.dat')

    preds_te = xgbr.predict(xtrain[idx_test])
    preds = np.concatenate([ytrain[idx_test].reshape(-1,1), preds_te.reshape(-1, 1), ytrain_[idx_test]], axis=1)
    preds_list.append(preds)

np.savez(file_path + 'cross_validation/NA_' + freq + '_predictions_' + model_sty + '.npz', cv0 = preds_list[0], cv01 = preds_list[1], cv02 = preds_list[2], cv3 = preds_list[3], cv4 = preds_list[4])

In [ ]:
results_cv, np.array(results_cv).mean(axis=0)

### Traing model in train sets

In [ ]:
# training model
result_ = model_xgbr_cv(xtrain, ytrain, xtest, ytest, verbosity=0, eta=0.19, num_round=295, gamma=0.25, colsample_bynode=1,
                        colsample_bytree=1, min_chlid_weight=0.5, alpha=1.9, lambdas=1.0)
# save model
xgbr = result_[6]
save_model_path = os.path.join(file_path_model, 'model', model_sty)
joblib.dump(xgbr, save_model_path + '/model_xgboost.dat')

### SHAP values

In [12]:
# load model
save_model_path = os.path.join(file_path_model, 'model', model_sty)
xgbr = joblib.load(save_model_path + '/model_xgboost.dat')

In [10]:
# expalin the model's predictions using SHAP， gputree
import shap
np.random.seed(100)
X = xtrain[np.random.choice(xtrain.shape[0], 500000, replace=True)]
explainer = shap.explainers.GPUTree(xgbr, X)

In [11]:
shap_values = explainer(X, check_additivity=False)

In [ ]:
# summarize the effects of all the features
shap.plots.beeswarm(shap_values, max_display=15)

In [ ]:
shap.plots.bar(shap_values, max_display=15)

In [12]:
# 存储 save shap_value 和 base value
np.save(file_path + 'shap_values/' + model_sty + '_shap_values_' + freq, shap_values.values)
np.save(file_path + 'shap_values/' + model_sty + '_base_values_' + freq, shap_values.base_values)

### Evaluation

In [ ]:
# define function of evaluation
def model_evaluation(feature, label, save_model_path):
    # 加载训练好的模型，对2020年数据进行预测，并将预测结果保存，写入数据库
    # load model
    model_ = joblib.load(save_model_path + '/model_xgboost.dat')
    # predicting
    predictions = model_.predict(feature)
    # load normalization paramaters
    norm_params = pd.read_csv(file_path + 'normalization_params/normalization_params_D_no2-idx-0')
    norm_std, norm_mean = norm_params['std'][0], norm_params['mean'][0]
    # 反标准化
    predictions_ = (predictions * sqrt(norm_std) + norm_mean).astype(int)
    label_ = (label * sqrt(norm_std) + norm_mean).astype(int)
    # metrics
    rmse = sqrt(mean_squared_error(label_, predictions_))
    mae = mean_absolute_error(label_, predictions_)
    r2 = r2_score(label_, predictions_)
    print('模型性能：', rmse, mae, r2)
    return predictions_, label_

pre_train, real_train = model_evaluation(xtrain, ytrain, save_model_path)
pre_test, real_test = model_evaluation(xtest, ytest, save_model_path)

### Save predictions

In [14]:
# load information of time and stations
data = np.load(file_path + 'numpy/NA_d_array_14.npz', allow_pickle=True, mmap_mode='r')
ytrain_, ytest_ = data['e'], data['f']

In [22]:
# 将预测数据和真实值匹配，并添加时间索引和站点名  : merge predictions and index
test_res = pd.concat([pd.DataFrame(real_test, columns=[list_col[0]]), pd.DataFrame(pre_test,columns=[col_name])
                         ,pd.DataFrame(ytest_[:,1:], columns=['index','station'])], axis=1)
train_res = pd.concat([pd.DataFrame(real_train,columns=[list_col[0]]),pd.DataFrame(pre_train,columns=[col_name])
                          ,pd.DataFrame(ytrain_[:,1:],columns=['index','station'])],axis=1)
data_res = pd.concat([train_res,test_res],axis=0)
data_res['index'] = (pd.to_datetime(data_res['index']) + pd.Timedelta('-1 days')).astype(str)

In [24]:
data_ = pd.read_csv(file_path + file_name_pre)

In [26]:
data_ = data_.drop(columns = col_name)

In [27]:
data_ = pd.merge(data_, data_res.drop(columns=list_col[0]), on=['index', 'station'], how='left')

In [30]:
# save predictions
data_.to_csv(file_path + file_name_pre, index=False)